In [2]:
import cv2
import pandas as pd
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [3]:
video_capture = cv2.VideoCapture(0)

clicked = False
r = g = b = x_pos = y_pos = 0

index = ["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)
def get_color_name(R, G, B):
    maximum = 10000
    for i in range(len(csv)):
        d = abs(R - int(csv.loc[i, "R"])) + abs(G - int(csv.loc[i, "G"])) + abs(B - int(csv.loc[i, "B"]))
        if d <= maximum:
            maximum = d
            cname = csv.loc[i, "color_name"]
    return cname
def locate_function(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global b, g, r, x_pos, y_pos, clicked
        clicked = True
        x_pos = x
        y_pos = y
        b, g, r = frame[y, x]
        b = int(b)
        g = int(g)
        r = int(r)


cv2.namedWindow('Color Detection')
cv2.setMouseCallback('Color Detection', locate_function)
while True:
    ret, frame = video_capture.read()

    cv2.imshow("Color Detection", frame)    
    if clicked:
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5, minSize=(200, 200))
        cv2.rectangle(frame, (20, 20), (620, 60), (b, g, r), -1)
        text = get_color_name(r, g, b) + ' R=' + str(r) + ' G=' + str(g) + ' B=' + str(b)
        cv2.putText(frame, text, (50, 50), 2, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 6)
            roi_color = frame[y:y + h, x:x + w]
            eyes = eye_cascade.detectMultiScale(roi_color, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
            for (ex, ey, ew, eh) in eyes:
            # convert eye coordinates to global coordinates
                ex_global, ey_global = x + ex, y + ey
                cv2.rectangle(frame, (ex_global, ey_global), (ex_global + ew, ey_global + eh), (0, 0, 255), 10)
        
        if r + g + b >= 600:
            cv2.putText(frame, text, (50, 50), 2, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.imshow("Color Detection", frame) 
        #clicked = False
#Esc
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()